In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

### Initialising a pre-trained YOLO model to detect people

In [2]:
person_model = YOLO("yolov8m.pt")

# persons = person_model("imagess.jpg", classes=[0])

### Loading our previously trained Bed Detection model

In [3]:
bed_model = YOLO("D:/runs/detect/train3/weights/best.pt")

##### Calculate the Intersection over Union (IoU) between two bounding boxes of two models

In [4]:
def iou(boxA, boxB):
    
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute intersection area
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Compute union area
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea

    return interArea / float(unionArea) if unionArea > 0 else 0

##### Detect beds and people, then determine if a person is on a bed.

In [5]:
def detect_bed_occupancy(media_path):

    bed_results = bed_model(media_path)
    person_results = person_model(media_path, classes=[0])  # Detect only perso

    # Extracting the bounding boxes
    bed_boxes = [list(map(int, box.xyxy[0])) for box in bed_results[0].boxes]
    person_boxes = [list(map(int, box.xyxy[0])) for box in person_results[0].boxes]
    
    img = cv2.imread(media_path)
    if img is None:
        print(f"Error: Could not load image : {media_path}")
        return 0
    img_Array = np.array(media_path)
    
    # Checking for overlaps between the lower section of a person and upper part of the bed bounding box
    occupied_beds = set()
    for bed_idx, bed in enumerate(bed_boxes):
        bed_x1, bed_y1, bed_x2, bed_y2 = bed
        for person in person_boxes:
            person_x1, person_y1, person_x2, person_y2 = person

            # Ignore standing people
            person_width = person_x2 - person_x1
            person_height = person_y2 - person_y1
            if person_height > 1.8 * person_width: 
                continue  

            # Check if lower half of person overlaps upper half of bed
            person_lower_half = [person_x1, person_y1 + int(person_height * 0.5), person_x2, person_y2]
            bed_upper_half = [bed_x1, bed_y1, bed_x2, bed_y1 + int((bed_y2 - bed_y1) * 0.5)]

            if iou(person_lower_half, bed_upper_half) > 0.2:  # Adjust threshold as needed
                occupied_beds.add(bed_idx)  # Store unique occupied beds
                cv2.rectangle(img, (bed_x1, bed_y1), (bed_x2, bed_y2), (0, 0, 255), 2)  # Red for occupied beds
                break  # Stop checking once the bed is occupied

    # Display image
    for box in bed_boxes:
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2) 
    for box in person_boxes:
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    print(f"Detected Beds: {len(bed_boxes)} | Detected Persons: {len(person_boxes)}")
    cv2.imshow("Bed Occupancy Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return len(occupied_beds)

### Testing the final product

In [7]:
detect_bed_occupancy("img2.jpg")


image 1/1 C:\Users\User\Documents\DS and ML\Bed-availability-Detection\img2.jpg: 384x640 1 hospital bed, 154.7ms
Speed: 0.0ms preprocess, 154.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\User\Documents\DS and ML\Bed-availability-Detection\img2.jpg: 384x640 3 persons, 144.1ms
Speed: 2.0ms preprocess, 144.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Detected Beds: 1 | Detected Persons: 3


1